Agentic AI

In [2]:
!pip install crewai langchain_groq streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.1/337.1 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 97.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 8.2 MB/s et

In [24]:
import streamlit as st
import pandas as pd
from datetime import datetime, timedelta
import json
import time
import os
from crewai import Agent, Task, Crew, Process
from crewai.tools import BaseTool
from langchain_groq import ChatGroq
from typing import Dict, List, Optional
import random


# Configuration
st.set_page_config(
    page_title="Doctor Appointment Booking Agent",
    page_icon="🏥",
    layout="wide"
)

# 💡 Just paste your API key directly here (don't hide it if running in Colab or Streamlit)
# GROQ_API_KEY = [YOUR GROQ API KEY]

# Optional 3-line secure alternative (for Hugging Face Spaces deployment):
import os
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
assert GROQ_API_KEY, "GROQ_API_KEY not set in environment"

# ✅ Cached LLM Loader for Streamlit
@st.cache_resource
def get_llm():
    return ChatGroq(
        temperature=0.1,
        groq_api_key=GROQ_API_KEY,
        model_name="groq/gemma2-9b-it",
    )


# Mock Database - In production, replace with actual database
MOCK_DOCTORS = [
    {
        "id": 1,
        "name": "Dr. Sarah Ahmed",
        "specialty": "Cardiology",
        "location": "Islamabad",
        "fee_pkr": 3000,
        "fee_usd": 11,
        "available_slots": ["2025-07-16 10:00", "2025-07-16 14:00", "2025-07-17 09:00"],
        "rating": 4.8,
        "experience": "15 years",
        "contact": "sarah.ahmed@hospital.com"
    },
    {
        "id": 2,
        "name": "Dr. Muhammad Hassan",
        "specialty": "Dermatology",
        "location": "Lahore",
        "fee_pkr": 2500,
        "fee_usd": 9,
        "available_slots": ["2025-07-16 11:00", "2025-07-16 15:00", "2025-07-17 10:00"],
        "rating": 4.6,
        "experience": "12 years",
        "contact": "hassan.derm@clinic.com"
    },
    {
        "id": 3,
        "name": "Dr. Fatima Khan",
        "specialty": "Pediatrics",
        "location": "Karachi",
        "fee_pkr": 2000,
        "fee_usd": 7,
        "available_slots": ["2025-07-16 09:00", "2025-07-16 13:00", "2025-07-17 11:00"],
        "rating": 4.9,
        "experience": "10 years",
        "contact": "fatima.peds@hospital.com"
    },
    {
        "id": 4,
        "name": "Dr. Ali Raza",
        "specialty": "Orthopedics",
        "location": "Rawalpindi",
        "fee_pkr": 3500,
        "fee_usd": 13,
        "available_slots": ["2025-07-16 12:00", "2025-07-16 16:00", "2025-07-17 08:00"],
        "rating": 4.7,
        "experience": "18 years",
        "contact": "ali.ortho@medical.com"
    },
    {
        "id": 5,
        "name": "Dr. Aisha Malik",
        "specialty": "Gynecology",
        "location": "Islamabad",
        "fee_pkr": 2800,
        "fee_usd": 10,
        "available_slots": ["2025-07-16 10:30", "2025-07-16 14:30", "2025-07-17 09:30"],
        "rating": 4.8,
        "experience": "14 years",
        "contact": "aisha.gyn@womens.com"
    }
]

# Custom Tools
class DoctorSearchTool(BaseTool):
    name: str = "doctor_search"
    description: str = "Search for doctors within specified budget range and specialty"

    def _run(self, specialty: str = "", max_budget_pkr: float = 5000, max_budget_usd: float = 18, location: str = "") -> str:
        """Search for doctors based on criteria"""
        matching_doctors = []

        for doctor in MOCK_DOCTORS:
            # Check budget constraints
            within_budget = (doctor["fee_pkr"] <= max_budget_pkr) or (doctor["fee_usd"] <= max_budget_usd)

            # Check specialty match
            specialty_match = not specialty or specialty.lower() in doctor["specialty"].lower()

            # Check location match
            location_match = not location or location.lower() in doctor["location"].lower()


            if within_budget and specialty_match and location_match:
                matching_doctors.append(doctor)

        if not matching_doctors:
            return "No doctors found matching your criteria."

        # Sort by rating and fee
        matching_doctors.sort(key=lambda x: (-x["rating"], x["fee_pkr"]))


        result = "Found matching doctors:\n"
        for doc in matching_doctors:
            result += f"- {doc['name']} ({doc['specialty']}) - {doc['location']}\n"
            result += f"  Fee: PKR {doc['fee_pkr']} / USD {doc['fee_usd']}\n"
            result += f"  Rating: {doc['rating']}/5.0, Experience: {doc['experience']}\n"
            result += f"  Available slots: {', '.join(doc['available_slots'])}\n\n"


        return result


class AppointmentBookingTool(BaseTool):
    name: str = "appointment_booking"
    description: str = "Book an appointment with a selected doctor"

    def _run(self, doctor_id: int, patient_name: str, patient_contact: str,
             preferred_slot: str, reason: str = "General consultation") -> str:
        """Book an appointment with the specified doctor"""


        # Find doctor
        doctor = None
        for doc in MOCK_DOCTORS:
            if doc["id"] == doctor_id:
                doctor = doc
                break

        if not doctor:
            return "Doctor not found."

        if preferred_slot not in doctor["available_slots"]:
            return f"Selected slot not available. Available slots: {', '.join(doctor['available_slots'])}"

        # Simulate booking process
        booking_id = f"BK{random.randint(1000, 9999)}"

        # Remove the booked slot from available slots
        doctor["available_slots"].remove(preferred_slot)

        booking_details = {
            "booking_id": booking_id,
            "doctor_name": doctor["name"],
            "doctor_specialty": doctor["specialty"],
            "patient_name": patient_name,
            "patient_contact": patient_contact,
            "appointment_slot": preferred_slot,
            "fee": f"PKR {doctor['fee_pkr']} / USD {doctor['fee_usd']}",
            "reason": reason,
            "status": "Confirmed",
            "booking_time": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }

        # Store booking in session state
        if "bookings" not in st.session_state:
            st.session_state.bookings = []
        st.session_state.bookings.append(booking_details)


        return f"""
        Appointment booked successfully!

        Booking ID: {booking_id}
        Doctor: {doctor['name']} ({doctor['specialty']})
        Patient: {patient_name}
        Contact: {patient_contact}
        Appointment: {preferred_slot}
        Fee: PKR {doctor['fee_pkr']} / USD {doctor['fee_usd']}
        Reason: {reason}
        Status: Confirmed

        You will receive a confirmation email shortly.
        """


class FormSubmissionTool(BaseTool):
    name: str = "form_submission"
    description: str = "Submit patient information form automatically"

    def _run(self, patient_data: Dict) -> str:
        """Submit patient form with provided data"""


        required_fields = ["name", "contact", "age", "gender", "medical_history"]
        missing_fields = [field for field in required_fields if field not in patient_data]


        if missing_fields:
            return f"Missing required fields: {', '.join(missing_fields)}"

        # Simulate form submission
        form_id = f"FORM{random.randint(1000, 9999)}"


        return f"""
        Patient form submitted successfully!

        Form ID: {form_id}
        Patient Name: {patient_data['name']}
        Contact: {patient_data['contact']}
        Age: {patient_data['age']}
        Gender: {patient_data['gender']}
        Medical History: {patient_data['medical_history']}

        Form has been processed and sent to the doctor.
        """


# Initialize LLM
llm = get_llm()


# Define Agents
def create_agents(llm):
    # Doctor Search Agent
    doctor_search_agent = Agent(
        role='Doctor Search Specialist',
        goal='Find the best doctors within the specified budget and criteria',
        backstory="""You are an expert medical appointment coordinator with extensive
        knowledge of healthcare providers. You specialize in finding the most suitable
        doctors based on patient requirements, budget constraints, and location preferences.""",
        tools=[DoctorSearchTool()],
        verbose=True,
        llm=llm
    )

    # Appointment Booking Agent
    booking_agent = Agent(
        role='Appointment Booking Specialist',
        goal='Book appointments efficiently and handle all booking logistics',
        backstory="""You are a skilled appointment booking specialist who ensures
        smooth scheduling processes. You handle all aspects of appointment booking
        including slot selection, patient information gathering, and confirmation.""",
        tools=[AppointmentBookingTool()],
        verbose=True,
        llm=llm
    )

    # Form Processing Agent
    form_agent = Agent(
        role='Form Processing Specialist',
        goal='Handle all form submissions and patient data processing',
        backstory="""You are a meticulous form processing specialist who ensures
        all patient information is accurately captured and submitted. You handle
        data validation, form completion, and submission processes.""",
        tools=[FormSubmissionTool()],
        verbose=True,
        llm=llm
    )

    return doctor_search_agent, booking_agent, form_agent


# Define Tasks
def create_tasks(search_agent, booking_agent, form_agent, user_requirements):
    # Task 1: Search for doctors
    search_task = Task(
        description=f"""
        Search for doctors based on these requirements:
        - Specialty: {user_requirements.get('specialty', 'Any')}
        - Location: {user_requirements.get('location', 'Any')}
        - Maximum budget: PKR {user_requirements.get('max_budget_pkr', 5000)} / USD {user_requirements.get('max_budget_usd', 18)}

        Provide a detailed analysis of available options with recommendations.
        """,
        agent=search_agent,
        expected_output="List of suitable doctors with their details, fees, and availability"
    )

    # Task 2: Book appointment
    booking_task = Task(
        description=f"""
        Based on the search results, book an appointment with the most suitable doctor.
        Patient details:
        - Name: {user_requirements.get('patient_name', 'Patient')}
        - Contact: {user_requirements.get('patient_contact', 'contact@email.com')}
        - Preferred time: {user_requirements.get('preferred_time', 'Morning')}
        - Reason: {user_requirements.get('reason', 'General consultation')}

        Select the best available slot and confirm the booking.
        """,
        agent=booking_agent,
        expected_output="Confirmed appointment booking with all details",
        context=[search_task]
    )

    # Task 3: Submit patient form
    form_task = Task(
        description=f"""
        Submit the patient information form with the following details:
        - Name: {user_requirements.get('patient_name', 'Patient')}
        - Contact: {user_requirements.get('patient_contact', 'contact@email.com')}
        - Age: {user_requirements.get('age', '30')}
        - Gender: {user_requirements.get('gender', 'Not specified')}
        - Medical History: {user_requirements.get('medical_history', 'None')}

        Ensure all required fields are properly filled and submitted.
        """,
        agent=form_agent,
        expected_output="Confirmation of successful form submission",
        context=[booking_task]
    )

    return search_task, booking_task, form_task

# Streamlit UI
def main():
    st.title("🏥 AI Doctor Appointment Booking Agent")
    st.markdown("Powered by CrewAI and Groq")

    # Check for API key from environment
    api_key = os.getenv('GROQ_API_KEY') or os.getenv('ChatBot_using_GroqAPI')
    api_key_available = bool(api_key)

    # API Key Notice
    if not api_key_available:
        st.error("🚫 Cannot proceed without API key. Please set GROQ_API_KEY as an environment variable.")

    # Sidebar for configuration
    with st.sidebar:
        st.header("🔧 Configuration")

        # API Key Status
        st.subheader("API Key Status")
        if api_key_available:
            st.success(f"🔑 API Key loaded successfully from environment.")
        else:
            st.error("🚫 No API key found. Please set GROQ_API_KEY.")

        # Setup Instructions
        st.subheader("📋 Setup Instructions")
        st.markdown("""
        **For Hugging Face or Colab:**
        - Use the environment variable: `GROQ_API_KEY` or fallback `ChatBot_using_GroqAPI`

        **In Colab:**
        ```python
        from google.colab import userdata
        os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
        ```

        **Manual Setup:**
        ```python
        import os
        os.environ['GROQ_API_KEY'] = 'your-api-key-here'
        ```

        🔗 [Get your API key from Groq](https://console.groq.com)
        """)

        # Model Info
        st.subheader("🤖 Model Info")
        st.info("Using: gemma2-9b-it via Groq")

    # Main interface
    col1, col2 = st.columns([2, 1])

    with col1:
        st.header("Appointment Booking Request")

        # Patient Information
        st.subheader("Patient Information")
        patient_name = st.text_input("Patient Name", value="John Doe")
        patient_contact = st.text_input("Contact (Email/Phone)", value="john.doe@email.com")
        age = st.number_input("Age", min_value=1, max_value=120, value=30)
        gender = st.selectbox("Gender", ["Male", "Female", "Other", "Prefer not to say"])
        medical_history = st.text_area("Medical History", value="No significant medical history")

        # Appointment Preferences
        st.subheader("Appointment Preferences")
        specialty = st.selectbox("Specialty", [
            "Any", "Cardiology", "Dermatology", "Pediatrics",
            "Orthopedics", "Gynecology", "Neurology", "Psychiatry"
        ])
        location = st.text_input("Preferred Location", value="Islamabad")

        # Budget
        st.subheader("Budget")
        currency = st.radio("Currency", ["PKR", "USD"])
        if currency == "PKR":
            max_budget = st.number_input("Maximum Budget (PKR)", min_value=1000, max_value=50000, value=3000)
            max_budget_pkr = max_budget
            max_budget_usd = max_budget / 275  # Approximate conversion
        else:
            max_budget = st.number_input("Maximum Budget (USD)", min_value=5, max_value=200, value=11)
            max_budget_usd = max_budget
            max_budget_pkr = max_budget * 275  # Approximate conversion

        # Additional preferences
        preferred_time = st.selectbox("Preferred Time", ["Morning", "Afternoon", "Evening", "Any"])
        reason = st.text_area("Reason for Visit", value="General consultation")

        # Start booking process
        if st.button("🤖 Start AI Booking Process", type="primary"):

            # Initialize LLM
            llm = get_llm()
            if not llm:
                st.error("❌ Failed to initialize Groq LLM")
                return

            # Prepare user requirements
            user_requirements = {
                'patient_name': patient_name,
                'patient_contact': patient_contact,
                'age': str(age),
                'gender': gender,
                'medical_history': medical_history,
                'specialty': specialty if specialty != "Any" else "",
                'location': location,
                'max_budget_pkr': max_budget_pkr,
                'max_budget_usd': max_budget_usd,
                'preferred_time': preferred_time,
                'reason': reason
            }

            # Create agents and tasks
            with st.spinner("Initializing AI agents..."):
                try:
                    search_agent, booking_agent, form_agent = create_agents(llm)
                    search_task, booking_task, form_task = create_tasks(
                        search_agent, booking_agent, form_agent, user_requirements
                    )
                except Exception as e:
                    st.error(f"Error creating agents: {e}")
                    return

            # Create and run crew
            crew = Crew(
                agents=[search_agent, booking_agent, form_agent],
                tasks=[search_task, booking_task, form_task],
                process=Process.sequential,
                verbose=True
            )

            # Execute the crew
            with st.spinner("AI agents are working on your booking..."):
                try:
                    result = crew.kickoff()
                    st.success("Booking process completed!")
                    st.text_area("AI Process Results", str(result), height=200)
                except Exception as e:
                    st.error(f"Error during booking process: {str(e)}")

    with col2:
        st.header("Available Doctors")

        # Display doctors
        for doctor in MOCK_DOCTORS:
            with st.expander(f"{doctor['name']} - {doctor['specialty']}"):
                st.write(f"**Location:** {doctor['location']}")
                st.write(f"**Fee:** PKR {doctor['fee_pkr']} / USD {doctor['fee_usd']}")
                st.write(f"**Rating:** {doctor['rating']}/5.0")
                st.write(f"**Experience:** {doctor['experience']}")
                st.write(f"**Available Slots:** {len(doctor['available_slots'])}")

        # Display bookings
        if "bookings" in st.session_state and st.session_state.bookings:
            st.header("Recent Bookings")
            for booking in st.session_state.bookings[-3:]:  # Show last 3 bookings
                with st.expander(f"Booking {booking['booking_id']}"):
                    st.write(f"**Doctor:** {booking['doctor_name']}")
                    st.write(f"**Patient:** {booking['patient_name']}")
                    st.write(f"**Appointment:** {booking['appointment_slot']}")
                    st.write(f"**Status:** {booking['status']}")

if __name__ == "__main__":
    main()

2025-07-15 10:54:38.089 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-15 10:54:38.103 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-15 10:54:38.104 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-15 10:54:38.106 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-15 10:54:38.108 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-15 10:54:38.109 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-15 10:54:38.110 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-15 10:54:38.110 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [25]:
!wget -q -O - ipv4.icanhazip.com

34.125.220.155


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

⠙

⠹⠸⠼⠴
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.220.155:8501

⠦⠧⠇⠏⠋your url is: https://clear-mugs-type.loca.lt


















╭─────────────────────────── Crew Execution Started ───────────────────────────╮
│                                                                              │
│  Crew Execution Started                                                      │
│  Name: crew                                                                  │
│  ID: b1d57cf4-2dff-4582-9b7d-95ff95b9a7bb                                    │
│  Tool Args:                                                                  │
│                                                                              │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯











🚀 Crew: cr